In [28]:
from bs4 import BeautifulSoup
import csv
from urllib.request import Request, urlopen, ProxyHandler, build_opener, install_opener, HTTPError
import pandas as pd
from random import randrange
from time import sleep
from json import loads
import requests
from IPython.display import Audio
from collections import defaultdict
import numpy as np

# Scraping FUTBIN for Name, Popularity Values

In [ ]:
## Getting User Agents and Proxies
def get_proxies():
    global https_proxies, http_proxies, user_agents
    req = requests.get('https://proxylist.geonode.com/api/proxy-list?limit=500&page=1&sort_by=lastChecked&sort_type=desc&protocols=http').json()['data']
    http_proxies = [r'http://' + req[q]["ip"] + f":{req[q]['port']}" for q in range(len(req))]
    req = requests.get('https://proxylist.geonode.com/api/proxy-list?limit=500&page=1&sort_by=lastChecked&sort_type=desc&protocols=https').json()['data']
    https_proxies = [r'https://' + req[q]["ip"] + f":{req[q]['port']}" for q in range(len(req))]
    user_agents = open("/content/drive/MyDrive/user_agents1.txt").readlines()
    user_agents = [i.strip() for i in user_agents]

#     req = loads(open('proxies.txt').read())['data']
#     proxies = [r'http://' + req[q]["ip"] + f":{req[q]['port']}" for q in range(len(req))]

    bruhh = []
    bruhs = []
    for i in range(1,5):
        try:
            req = Request(
                url=f'https://hidemy.name/en/proxy-list/?type=h&start={64*i}#list', 
                headers={'User-Agent': user_agents[randrange(0,len(user_agents))]}
            )
            webpage = urlopen(req)#, proxies={'http':proxies[randrange(0, len(proxies))]}).read()
            x = pd.read_html(webpage)[0]
            bruhh.append(x)#[["IP address"], ["Port"]])#[["IP address"], ["Port"]])

        except Exception as e:
            break        
          
    bruhh = pd.concat(bruhh)
    for i in range(bruhh.shape[0]):
        http_proxies.append("http://" + bruhh["IP address"].iloc[i] + ':' + f'{bruhh["Port"].iloc[i]}')


    for i in range(1,5):
        try:
            req = Request(
                url=f'https://hidemy.name/en/proxy-list/?type=s&start={64*i}#list', 
                headers={'User-Agent': user_agents[randrange(0,len(user_agents))]}
            )
            webpage = urlopen(req)#, proxies={'http':proxies[randrange(0, len(proxies))]}).read()
            x = pd.read_html(webpage)[0]
            bruhs.append(x)#[["IP address"], ["Port"]])#[["IP address"], ["Port"]])

        except Exception as e:
            break 

    bruhs = pd.concat(bruhs)
    for i in range(bruhs.shape[0]):
        https_proxies.append("https://" + bruhs["IP address"].iloc[i] + ':' + f'{bruhs["Port"].iloc[i]}')  

    
get_proxies()

In [ ]:
columns = [
 #'name',
 'rat',
 'position',
 'pace',
 'acceleration',
 'sprintspeed',
 'shooting',
 'positioning',
 'finishing',
 'shotpower',
 'longshotsaccuracy',
 'volleys',
 'penalties',
 'passing',
 'vision',
 'crossing',
 'freekickaccuracy',
 'shortpassing',
 'longpassing',
 'curve',
 'dribblingp',
 'agility',
 'balance',
 'reactions',
 'ballcontrol',
 'dribbling',
 'composure',
 'defending',
 'interceptions',
 'headingaccuracy',
 'marking',
 'standingtackle',
 'slidingtackle',
 'heading',
 'jumping',
 'stamina',
 'strength',
 'aggression',
 'traits',
 'xbox_gp',
 'ps4_gp',
 'pop'
]
    
def get_players(req,year):
    bs = BeautifulSoup(req)
    def v(s):
        if type(s) == str:
            return f"/{year}/player/" in s
        return False
    players = bs.find_all(href=v)
    return players

def get_stats(req):
    bs = BeautifulSoup(req)
    stats1 = bs.find_all("div", class_="stat_val")
    votes = bs.find_all("span", class_="total_votes")
    v1 = (int(votes[1].contents[0]))
    v0 = (int(votes[0].contents[0]))
    votes = v1 - v0
    pos = bs.find("div", class_="pcdisplay-pos")
    rat = bs.find("div", class_="pcdisplay-rat")
    ps4_gp = bs.find_all("div", class_="ps4-pgp-data")
    ps4_gp = ps4_gp[-1].contents[0].strip()
    
    xbox_gp = bs.find_all("div", class_="xbox-pgp-data hide")
    xbox_gp = xbox_gp[-1].contents[0].strip()
    traits = [i.contents[0] for i in bs.find_all("div", class_="trait-name-val")]
    return stats1,votes,pos,ps4_gp,xbox_gp,traits,rat
 

years = [22,21,20,19,18,17,16]
ks = [643,598,696,600,654,635,638]
for year,pages in zip(years,ks):
    get_proxies()
    p = 1
    final = []
    while p != pages:
        print(p)
        try:
            req0 = urlopen(Request(
                            url=f"https://www.futbin.com/{year}/players?page={p}&version=all_nif",
                            headers={'User-Agent': user_agents[randrange(0,len(user_agents))],
                                    'referer': 'https://www.google.com'
                                    }
                        ))
            plinks = get_players(req0,year)
            for l in plinks:
                while True:
                    sl = []
                    try:
                        url = "https://www.futbin.com/" + l.get("href")
                        req1 = urlopen(Request(
                            url,
                            headers={'User-Agent': user_agents[randrange(0,len(user_agents))],
                                    'referer': 'https://www.google.com'
                                    }
                        )).read()
                        s = pd.read_html(req1,index_col=0)[0]
                        stats1,votes,pos,ps4_gp,xbox_gp,traits,rat = get_stats(req1)
                        sl.append(rat.contents[0])
                        sl.append(pos.contents[0])
                        if pos.contents[0] != "GK" or year < 19:
                            for i in range(0,len(stats1)-1,2):
                                sl.append(stats1[i+1].contents[0])
                        else:
                            for i in range(26,len(stats1)-1,2):
                                sl.append(stats1[i+1].contents[0])
                        sl+= [traits,ps4_gp,xbox_gp,votes]
                        sl = pd.Series(sl, index=columns)
                        sl = pd.concat([s[1],sl])
                        if type(final) == list:
                          if len(final) == 1:
                            final.append(sl)
                            final = pd.DataFrame(final)
                            final.to_csv(f"/content/drive/MyDrive/FIFA/futbin-{year}-big.csv", index=False)
                          elif len(final) == 0:
                            final.append(sl)
                          else:
                            print(final)
                        else:
                          sl = sl.to_frame().transpose()
                          sl.to_csv(f"/content/drive/MyDrive/FIFA/futbin-{year}-big.csv", mode='a', header=False, index=False)
                        break
                    except Exception as e:
                        print(e)
                        sleep(5)
                        next
            p+=1
        except Exception as e:
            print(e)
            sleep(5)
            next